<a href="https://colab.research.google.com/github/cyyeh/ds-portfolios/blob/master/projects/youbike/notebooks/1.0-youbike-weather-air-data-integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Youbike, Weather, Air Data Integration

In [0]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Load Preprocessed Data

- youbike data: https://github.com/cyyeh/ds-portfolios/raw/master/projects/youbike/data/interim/1.0-youbike-history-data-186-df.pkl
- weather-air data: https://github.com/cyyeh/ds-portfolios/raw/master/projects/youbike/data/interim/1.0-weather-air-history-data-taipei-df.pkl

In [0]:
def get_weather_air_df(path='drive/My Drive/1.0-weather-air-history-data-taipei-df.pkl'):
  return pd.read_pickle(path)

In [0]:
def get_youbike_df(path='drive/My Drive/1.0-youbike-history-data-186-df.pkl'):
  return pd.read_pickle(path)

In [0]:
def get_youbike_integration_df(youbike_df=None, weather_air_df=None):
    if not youbike_df:
        youbike_df = get_youbike_df()
    if not weather_air_df:
        weather_air_df = get_weather_air_df()

    # merge two dataframes and do some preprocessings
    # - imputation
    # - filter out redundant fields
    # - rename columns
    fields_needed = [
        '可借車數',
        '星期幾',
        '是否為週末',
        '細懸浮微粒(μg/m^3)',
        '總碳氫化合物(ppm)',
        '懸浮微粒(μg/m^3)',
        '甲烷(ppm)',
        '一氧化碳(ppm)',
        '二氧化氮(ppb)',
        '氮氧化物(ppb)',
        '二氧化硫(ppb)',
        '臭氧(ppb)',
        '氣溫(℃)',
        '相對溼度(%)',
        '風速(m/s)',
        '風向(360degree)',
        '降水量(mm)'
    ]

    rename_mapper = {
        'current_number': '可借車數',
        'weekday': '星期幾',
        'is_weekend': '是否為週末'
    }

    df = (pd.merge(youbike_df, weather_air_df,
                   how='outer', left_index=True,
                   right_index=True, sort=True)
            .rename(columns=rename_mapper)
          )

    df[fields_needed[1:]] = df[fields_needed[1:]].fillna(method='ffill')
    df = df.dropna() # remove rows with NaN inside the column of 可借車數

    # weekday mapper
    weekday_dict = {
        0.0: 'Mon',
        1.0: 'Tue',
        2.0: 'Wed',
        3.0: 'Thu',
        4.0: 'Fri',
        5.0: 'Sat',
        6.0: 'Sun'
    }
    df['星期幾'] = df['星期幾'].apply(lambda weekday: weekday_dict[weekday]).astype('category')

    return df[fields_needed]

In [0]:
df = get_youbike_integration_df()

In [28]:
df

,可借車數,星期幾,是否為週末,細懸浮微粒(μg/m^3),總碳氫化合物(ppm),懸浮微粒(μg/m^3),甲烷(ppm),一氧化碳(ppm),二氧化氮(ppb),氮氧化物(ppb),二氧化硫(ppb),臭氧(ppb),氣溫(℃),相對溼度(%),風速(m/s),風向(360degree),降水量(mm)
2018-01-01 00:10:00,29.0,Mon,False,12.0,2.0,80.0,1.9,0.48,16.0,17.0,1.7,32.0,15.9,71.0,3.3,50.0,0.0
2018-01-01 00:15:00,29.0,Mon,False,12.0,2.0,80.0,1.9,0.48,16.0,17.0,1.7,32.0,15.9,71.0,3.3,50.0,0.0
2018-01-01 00:20:00,30.0,Mon,False,12.0,2.0,80.0,1.9,0.48,16.0,17.0,1.7,32.0,15.9,71.0,3.3,50.0,0.0
2018-01-01 00:25:00,24.0,Mon,False,12.0,2.0,80.0,1.9,0.48,16.0,17.0,1.7,32.0,15.9,71.0,3.3,50.0,0.0
2018-01-01 00:30:00,23.0,Mon,False,12.0,2.0,80.0,1.9,0.48,16.0,17.0,1.7,32.0,15.9,71.0,3.3,50.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-15 23:40:00,3.0,Fri,False,3.0,2.5,22.0,2.0,0.91,39.0,50.0,4.1,2.9,24.6,81.0,1.2,170.0,0.1
2018-06-15 23:45:00,4.0,Fri,False,3.0,2.5,22.0,2.0,0.91,39.0,50.0,4.1,2.9,24.6,81.0,1.2,170.0,0.1
2018-06-15 23:50:00,3.0,Fri,False,3.0,2.5,22.0,2.0,0.91,39.0,50.0,4.1,2.9,24.6,81.0,1.2,170.0,0.1
2018-06-15 23:55:00,0.0,Fri,False,3.0,2.5,22.0,2.0,0.91,39.0,50.0,4.1,2.9,24.6,81.0,1.2,170.0,0.1


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 43788 entries, 2018-01-01 00:10:00 to 2018-06-16 00:00:00
Data columns (total 17 columns):
可借車數             43788 non-null float64
星期幾              43788 non-null category
是否為週末            43788 non-null object
細懸浮微粒(μg/m^3)    43788 non-null float64
總碳氫化合物(ppm)      43788 non-null float64
懸浮微粒(μg/m^3)     43788 non-null float64
甲烷(ppm)          43788 non-null float64
一氧化碳(ppm)        43788 non-null float64
二氧化氮(ppb)        43788 non-null float64
氮氧化物(ppb)        43788 non-null float64
二氧化硫(ppb)        43788 non-null float64
臭氧(ppb)          43788 non-null float64
氣溫(℃)            43788 non-null float64
相對溼度(%)          43788 non-null float64
風速(m/s)          43788 non-null float64
風向(360degree)    43788 non-null float64
降水量(mm)          43788 non-null float64
dtypes: category(1), float64(15), object(1)
memory usage: 5.7+ MB


In [0]:
df.to_pickle('1.0-youbike-integration-df.pkl')